In [ ]:
# @title ## 1️⃣ Install
# @markdown Install required files and dependencies.
%%capture
!git clone https://github.com/TencentARC/GFPGAN
%cd GFPGAN
!pip install -r requirements.txt
!pip install basicsr
!pip install facexlib
!pip install realesrgan
!python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
# @title ## 2️⃣ Upload Video
# @markdown Upload your **MP4** video file.

# @markdown Note: We recommend uploading a video with a resolution not exceeding **480p** and a duration not exceeding **5 minutes**. Videos that exceed these specifications may take hours to upscale.
from google.colab import files, output
from IPython.display import HTML
import os
import cv2

def is_valid_video(file_path):
    try:
        cap = cv2.VideoCapture(file_path)
        return cap.isOpened()
    except:
        return False
    finally:
        if cap.isOpened():
            cap.release()

def print_error(message):
    html_message = f'<pre><span style="color: var(--ansi-red);">Error</span><span>: {message}</span></pre>'
    display(HTML(html_message))

uploaded = files.upload()

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() == 'mp4'

if len(uploaded) == 1:
    uploaded_filename = list(uploaded.keys())[0]
    if allowed_file(uploaded_filename) and is_valid_video(uploaded_filename):
        destination_directory = '/content/sample_data'
        new_filename = 'input.mp4'
        new_filepath = os.path.join(destination_directory, new_filename)
        os.rename(uploaded_filename, new_filepath)
    else:
        output.clear()
        print_error('Please upload a valid MP4 video file.')
        raise Exception()
else:
    output.clear()
    print_error('Please upload only one video file at once.')
    raise Exception()

In [ ]:
# @title ## 3️⃣ Upscale Video
# @markdown Upscale your video file.
%%capture
Scale = '2' # @param ["2", "3"]
# @markdown Note: A 480p video file with a 5-second duration takes around 5 minutes to complete.

inputFramesPath = '/content/GFPGAN/inputs/frames'
!rm -rf $inputFramesPath
!mkdir $inputFramesPath
from tqdm import tqdm
from os import path
#import os
vidcap = cv2.VideoCapture("/content/sample_data/input.mp4")
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)
for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('/content/GFPGAN/inputs/frames', str(frameNumber).zfill(4)+'.jpg'), image)
!rm -rf results/restored_imgs
!python inference_gfpgan.py -i /content/GFPGAN/inputs/frames -s {Scale}

In [ ]:
# @title ## 4️⃣ Download Video
# @markdown Download your upscaled video.
upscaledFramesPath = '/content/GFPGAN/results/restored_imgs/'
upscaledVideoPath = '/content/upscaled-video-vidscale-by-howover.com.mp4'
import numpy as np
dir_list = os.listdir(upscaledFramesPath)
dir_list.sort()
batch = 0
batchSize = 600
img_array = []
for i in range(0, len(dir_list), batchSize):
    start, end = i, i + batchSize
    for filename in dir_list[start:end]:
        filename = upscaledFramesPath + filename
        img = cv2.imread(filename)
        if img is not None:
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)
out = cv2.VideoWriter(upscaledVideoPath, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
batch = batch + 1
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
files.download(upscaledVideoPath)
thanks = '<pre><span>Your video is being downloaded, thank you for using <b><a href="https://howover.com/vidscale" target="_blank" style="text-decoration: none;">VidScale</a></b></span>'
display(HTML(thanks))

---
>**Was VidScale helpful?** If so, you can [<button>Donate a Coffee</button>](https://buymeacoffee.com/howover)
---
**Credits**: This notebook is prepared, published, and maintained by [Howover](https://howover.com/vidscale), and we would like to thank the hard work of open-source developers and contributors behind the following:
* Real-ESRGAN by xinntao for providing an open-source algorithm for video restoration;
* BasicSR by xinntao for providing an open-source video restoration toolbox;
* GFPGAN by xinntao for providing an open-source algorithm for face restoration;
* FaceXLib by xinntao for providing open-source face-related functions;
* Colab by Google for offering free GPU resources; and
* Everyone or anyone who is directly or indirectly involved.

If you still have any questions, please don't hesitate to [contact us](https://howover.com/contact).

**Version**: 1.0 Beta